## DeFi

In [ ]:
import hashlib # hash 함수용 sha256 사용할 라이브러리
import json
from time import time
import random
import requests
from flask import Flask, request, jsonify
import datetime

In [ ]:
#Staking Token 정의

token_name = "DuHyeon-STAKING-TOKEN"
token_total_volume = 100000
token_owner = {'토큰_발행자', 10000}
staking_status = {}

In [ ]:
#Token의 기본 기능 설정

def get_balance(user_id):
    print('{} Balance is : '.format(user_id), token_owner[user_id])
    return token_owner[user_id]

def send_token(sender, recipent, amount):
    if sender in token_owner.keys():
        if get_balance(sender) > amount:
            token_owner[sender] = token_owner[sender] - amount
            if recipent in token_owner.keys():
                token_owner[recipent] = token_owner[recipent] + amount
            else:
                token_owner[recipent] = amount
            print("트랜젝션 완료")
            get_balance(sender)
            get_balance(recipent)
        else:
            return "잔액부족"
    else:
        return "사용할 수 없는 발신자 ID"

In [ ]:
# 토큰 예치(Staking) 기능 설정

def token_staking(staker, amount):
    if staker in token_owner.keys():
        if get_balance(staker) > amount:
            token_owner[staker] = token_owner[staker] - amount
            staker [len(staking_status)] = {'staker' : staker, 'amount': amount}
            print("스테이킹 완료")
            get_balance(staker)
        else:
            return "잔액부족"
    else:
        return "사용할 수 없는 발신자 ID"
    
def staking_yield(staking_status):
    for t in staking_status:
        print(staking_status[t])
        staking_status[t]['amount'] = staking_status[t]['amount'] * (1+0.1)
    return staking_status

In [ ]:
# (Defi) DAPP의 스마트 컨트랙트

headers = {'Content-Type' : 'application/json; charset = utf-8'}
data = {
    "sender" : "test_from",
    "recipient": "test_to",
    "amount" : 3,
    "smart_contract" : {
        "contract_code" : "token_name = 'DuHyeon-STAKING-TOKEN' \ntoken_total_volume=100000\ntoken_owner = {'토큰_발행자' : 100000}\nstaking_status = {}",
        "contract_function_getBalance" :'''
def get_balance(user_id):
    print('{} Balance is : '.format(user_id), token_owner[user_id])
    return token_owner[user_id]''', "contract_function_sendToken" : """
def send_token(sender, recipent, amount):
    if sender in token_owner.keys():
        if get_balance(sender) > amount:
            token_owner[sender] = token_owner[sender] - amount
            if recipent in token_owner.keys():
                token_owner[recipent] = token_owner[recipent] + amount
            else : 
                token_owner[recipent] = amount
                print("Transaction Completed")
                get_balance(sender)
                get_balance(recipent)
        else:
            return "잔액부족"
    else:
        return "사용할 수 없는 발신자 ID"
""",
    "contract_function_token_staking" :"""
def token_staking(staker,amount):
    if staker in token_owner.keys():                       ## 예치자(staker)가 실제 존재하는 사용자인지 확인
        if get_balance(staker) > amount:                   ## 예치자(staker)의 잔고가 예치 금액보다 많은지 확인
            token_owner[staker]  = token_owner[staker] - amount   ## 예치자(staker)의 잔고에서 예치 금액 제외
            staking_status [len(staking_status)] =  {'staker':staker,'amount':amount}  
                                                            ## 예치 정보(staking_status)에 예치자(staker)의 예치내역 저장
            print("스테이킹 완료")
            get_balance(staker) 
            
        else:
            return "잔액부족"
    else:
        return "사용할 수 없는 발신자 ID"
""",
    "contract_function_staking_yield" :"""
def staking_yield(staking_status):                                 ## 예치 이자 지급함수
    for t in staking_status:
        print(staking_status[t])
            staking_status[t]['amount'] = staking_status[t]['amount'] * (1+0.1)    ## 예치 이자가 10% 지급된 금액으로 예치금 변경
    return staking_status
"""                       

    }  
}
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']


headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

## 생성된 토큰 확인하기

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])
            break

In [ ]:
token_name

In [ ]:
token_total_volume

In [ ]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('토큰_발행자')

## 예치 진행

In [ ]:
exec(_tx['smart_contract']['contract_function_token_staking'])
token_staking('토큰_발행자', 100)

In [ ]:
exec(_tx['smart_contract']['contract_function_token_staking'])
token_staking('토큰_발행자',1000)

In [ ]:
#staking_yield 함수로 예치 이자를 지급

exec(_tx['smart_contract']['contract_function_staking_yield'])
staking_yield(staking_status)

In [ ]:
exec(_tx['smart_contract']['contract_function_staking_yield'])
staking_yield(staking_status)

In [ ]:
exec(_tx['smart_contract']['contract_function_staking_yield'])
staking_yield(staking_status)